# Fördjupad Pythonprogrammering

## Kunskapskontrol 1

## Lidiia Kashevarova 

# Fråga 1

Koden konfigurerar ett grafiskt användargränssnitt (GUI) med hjälp av Tkinter för att visa och interagera med data från en CSV-fil. Användaren kan justera bredden på kolumnerna för att förbättra visningen av data och sedan exportera den till en ny CSV-fil. Dessutom inkluderar applikationen felhantering. 

In [ ]:
import pandas as pd # Imports the Pandas library
import tkinter as tk # Imports the tkinter library, which is used for creating graphical user interfaces (GUIs) in Python
from tkinter import ttk, filedialog, messagebox # Imports specific components from tkinter:
# - `ttk`: A themed widget set for tkinter, providing a modern look.
# - `filedialog`: Provides functions to open file dialogs, allowing users to choose files from their file system.
# - `messagebox`: Allows displaying pop-up dialog boxes for messages like warnings or confirmations.

# Läs in CSV-filen
df = pd.read_csv('test_data.csv')# Reads the CSV file

class VehicleViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Autostrada - CSV Viewer")
        self.filtered_df = df.copy()  # Save filtered dataframe here

        # Set window icon
        self.set_window_icon()

        # Style configurations
        style = ttk.Style()
        style.configure("Treeview", rowheight=30, font=('Arial', 10))# Sets the row height and font for the table
        style.configure("Treeview.Heading", font=('Arial', 12, 'bold')) # Configures the table headings
        style.configure("Treeview", background="#F5F5F5", foreground="#000000", fieldbackground="#FFFFFF")# Table background and foreground colors
        style.configure('TButton', font=('Arial', 10), padding=6)# Configures the button style

        # Create a main frame
        main_frame = tk.Frame(root, bg="#EAEAEA")
        main_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)

        # Create filter frame
        filter_frame = tk.Frame(main_frame, bg="#EAEAEA")
        filter_frame.grid(row=0, column=0, sticky="ew")

        # Add a label and entry for filtering
        self.filter_label = tk.Label(filter_frame, text="Filter (Column:Value):", bg="#EAEAEA")# frame where the label will appear and the text that will appear in the label
        self.filter_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")

        self.filter_entry = tk.Entry(filter_frame, width=40) # Text input for filter criteria
        self.filter_entry.grid(row=0, column=1, padx=5, pady=5)

        self.filter_button = tk.Button(filter_frame, text="Apply Filter", command=self.apply_filter) # Button to apply the filter
        self.filter_button.grid(row=0, column=2, padx=5, pady=5)

        # Result Display as a table
        self.tree = ttk.Treeview(main_frame, columns=[col for col in df.columns], show='headings') # Create a table to display CSV data
        self.tree.grid(row=1, column=0, padx=10, pady=10, sticky='nsew')

        # Scrollbars
        self.vsb = ttk.Scrollbar(main_frame, orient="vertical", command=self.tree.yview)  # Add vertical scrollbars for the table
        self.vsb.grid(row=1, column=1, sticky='ns')
        self.tree.configure(yscrollcommand=self.vsb.set)

        self.hsb = ttk.Scrollbar(main_frame, orient="horizontal", command=self.tree.xview)#Add horizontal scrollbars for the table
        self.hsb.grid(row=2, column=0, columnspan=2, sticky='ew')
        self.tree.configure(xscrollcommand=self.hsb.set)

        # Set column headings and initial column width
        for col in df.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100, stretch=tk.NO)  # Initial width, stretch turned off

        self.auto_adjust_column_width()  # Automatically adjust the column widths based on content


        # Row count label
        self.row_count_label = tk.Label(main_frame, text=f"Antal rader: {len(df)}", font=('Arial', 10, 'bold'), bg="#EAEAEA")
        self.row_count_label.grid(row=2, column=0, pady=10, sticky="w")

        # Configure grid weights
        main_frame.grid_rowconfigure(1, weight=1)
        main_frame.grid_columnconfigure(0, weight=1)

        # Make the main window resize with the content
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Display the full data
        self.display_results(self.filtered_df)

    def set_window_icon(self):
        try:
            # Load and set the window icon
            self.icon_image = tk.PhotoImage(file='icon.png')  # Path to your icon file
            self.root.iconphoto(True, self.icon_image) # Set the icon for the window
        except tk.TclError:
            # Handle error if icon cannot be loaded
            print("Error loading icon image. Make sure the file path is correct and the file is a valid image format.")

    def auto_adjust_column_width(self):
        # Create a label for text measurement
        label = tk.Label(self.root, font=('Arial', 10))
        
         # Initialize max_width dictionary to store max width for each column
        max_width = {col: len(self.tree.heading(col, 'text')) * 10 for col in self.tree["columns"]}  

        # Adjust width across all lines
        for item in self.tree.get_children():
            row_values = self.tree.item(item, 'values')
            for col, value in zip(self.tree["columns"], row_values):
                label.config(text=value)
                text_width = label.winfo_reqwidth()
                max_width[col] = max(max_width[col], text_width)

        # Set column width based on the widest content
        for col, width in max_width.items():
            self.tree.column(col, width=width + 20)  # Add extra padding

    def display_results(self, df_to_display):
        # Clear previous results
        for item in self.tree.get_children():
            self.tree.delete(item)

        # Display results
        for _, row in df_to_display.iterrows():
            self.tree.insert("", "end", values=row.tolist())

    def apply_filter(self):
        filter_text = self.filter_entry.get() # Get the filter text from input
        if not filter_text:
            self.filtered_df = df.copy()  # If filter is empty, reset to full DataFrame
        else:
            try: 
                #Parse the filter input as 'Column:Value'
                column, value = filter_text.split(':', 1)
                column = column.strip() # Remove any extra spaces
                value = value.strip()
                
                 # Apply filter if column exists in the DataFrame
                if column in df.columns:
                    self.filtered_df = df[df[column].astype(str).str.contains(value, case=False, na=False)]
                else:
                    messagebox.showwarning("Invalid Column", f"Column '{column}' does not exist.")
                    self.filtered_df = df.copy()
            except ValueError:
                messagebox.showwarning("Invalid Filter", "Filter format should be 'Column:Value'.")
                self.filtered_df = df.copy()

        # Update the display with filtered data
        self.display_results(self.filtered_df)
        self.row_count_label.config(text=f"Antal rader: {len(self.filtered_df)}")# Update row count

    def export_to_csv(self):
        # Ask user for a file name and location
        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                 filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if not file_path:
            return  # User canceled the save dialog

        try:
            # Export DataFrame to CSV with UTF-8 encoding
            self.filtered_df.to_csv(file_path, index=False, encoding='utf-8-sig')
            messagebox.showinfo("Export Success", "Data exported successfully!")
        except Exception as e:
            messagebox.showerror("Export Error", f"An error occurred while exporting the data:\n{e}")


# Create the application window
root = tk.Tk() # Tkinter window
app = VehicleViewerApp(root) #VehicleViewerApp
root.mainloop() # Start the loop 

![Result](f_1.png)

# Fråga 2

## Kapitel 6

1. Kolla vilka klasser variablerna a, b, c, och d tillhör genom att använda type() funktionen.
   Notera att vi i variablen d har sparat en funktion. Vi hade kunnat anropa den genom att till exempel skriva d(2). 

In [143]:
a = 10 
b = [5, 7, 3] 
c = {2, 7, 1, 8, 2, 8}

def my_fun(x):
    return 2*2

d = my_fun

In [144]:
print(type(a))
print(type(b))
print(type(c))
print(type(d))
e=d(2)
print(e)

<class 'int'>
<class 'list'>
<class 'set'>
<class 'function'>
4


2. a) Kolla om variablen "my_variable" är en instans av "tuple" klassen genom att använda isinstance() funktionen.
   
   b) Kolla om variablen "my_variable" är en instans av "list" klassen genom att använda isinstance() funktionen.

In [145]:
my_variable = (1, 1, 2, 3, 5)

In [146]:
is_tuple = isinstance(my_variable, tuple)
print(is_tuple) 

True


In [147]:
is_list = isinstance(my_variable, list)
print(is_list) 

False


3. I denna uppgiften skall du göra följande:
 
   a) Skapa en klass som heter "FruitProduct" som har en docstring där det står "A class representing fruit products in a grocery store.".
   
   b) Klassen skall ha instans-attributen "price" och "quantity". Detta gör du i samband med _\_\_init_\__() .
   
   c) Instantiera klassen och spara instansen i variablen "swedish_apples". Dess pris skall vara 52 och dess kvantitet 1. Det är underförstått att vi menar kronor respektive kg. 
   
   d) Printa ut klassens docstring genom att använda dig av _\_\_doc_\_\_.
   
   e) Printa ut attributen från "swedish_apples" instansen.
   
   f) Kolla vilken klass instansen "swedish_apples" tillhör genom att använda type() metoden.
   
   g) Kolla om "swedish_apples" tillhör klassen "FruitProduct" genom att använda isinstance() metoden.

In [148]:
class FruitProduct:
    """A class representing fruit products in a grocery store."""
    
    def __init__(self, price, quantity):
        self.price=price
        self.quantity=quantity
        
swedish_apples=FruitProduct(52, 1) 

print(FruitProduct.__doc__)
print(f'Price: {swedish_apples.price} per  Quantity: {swedish_apples.quantity} kg')


A class representing fruit products in a grocery store.
Price: 52 per  Quantity: 1 kg


In [149]:
type(swedish_apples)

__main__.FruitProduct

In [150]:
isinstance(swedish_apples,FruitProduct)

True

4. Skapa en klass "Square" vars instanser har attributet "side_length". Skapa två metoder, "perimeter()" och "area()" där metoderna beräknar omkretsen respektive arean givet längden på kvadraternas sidor som finns i attributet "side_length". Skapa en instans "my_square" med sidlängden 8. Beräkna omkretsen och arean av "my_square" genom att använda de två metoderna som du har skapat. 

In [151]:
class Sguare:
    
    def __init__(self, side_length):
        self.side_length=side_length
    
    def perimetr(self):
        return self.side_length*4
      
    
    def area(self):
        return self.side_length**2

In [152]:
my_sguare=Sguare(5)

print(f'Perimetr: {my_sguare.perimetr()} and area: {my_sguare.area()}')

Perimetr: 20 and area: 25


5. a) Förklara vad nedanstående kod gör.
   
   b) Prova ändra variabeln L i koden nedan till en tuple och se vad som sker när du kör om all kod. 

In [153]:
class DescriptiveStatistics():
    """This class provides functionality for calculating descriptive statistics from a list.""" #brief descriptions of what the class does
    
    def __init__(self):  #method constructor is used when a new instance of the class is created
        self.data = []
    
    def add_data(self, data): #method takes the data and adds it to the list self.data
        if isinstance(data, list): #checks if the data is a list
            self.data.extend(data) # If the data is a list, its contents are appended to self.data using self.data.extend(data).
        else:
            # Code below raises an error if the data is not a list. Will be covered in chapter 8 of the book. 
            raise Exception('Only "Lists" are accepted as data.')#If the data is not a list, the method throws an Exception with an error message
            
    def calc_sum(self): #method returns the sum of all elements in self.data
        return sum(self.data) #sum() is Python-funktionen
    
    def calc_nbr_of_elements(self): #method returns the number of all elements in self.data
        return len(self.data)  #method returns the numberfunktionen
    
    def calc_mean(self): #method calculates and returns the mean  of all elements i self.data 
        return (self.calc_sum())/(self.calc_nbr_of_elements())
    
    def print_summary(self):
        print('Sum:', self.calc_sum())
        print('Number of elements:', self.calc_nbr_of_elements())
        print('Mean:', self.calc_mean())  

In [154]:
L = [1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1]
my_data = DescriptiveStatistics()
my_data.add_data(L)

In [155]:
my_data.data

[1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1]

In [156]:
print('Sum:', my_data.calc_sum())
print('Number of elements:', my_data.calc_nbr_of_elements())
print('Mean:', my_data.calc_mean())

Sum: 105
Number of elements: 25
Mean: 4.2


In [157]:
my_data.print_summary()

Sum: 105
Number of elements: 25
Mean: 4.2


In [158]:
L = (1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1)
type(L)

tuple

In [159]:
my_data = DescriptiveStatistics()
my_data.add_data(L)

Exception: Only "Lists" are accepted as data.

6. a) Skapa en klass som heter BankAccount. Klassen skall ha attributet "account_holder" som visar kontoinnehavarens namn samt attributet "balance" som visar kontoinnehavarens balans. Klassen skall ha metoden "deposit()" för att kunna sätta in pengar på kontot samt metoden "withdraw()" för att kunna ta ut pengar från kontot. Om bankinnehavaren försöker ta ut mer pengar än vad som finns på kontot skall meddelandet "Too low balance" printas ut.
   
   b) Skapa en instans av klassen och testa så klassen funkar så som du förväntar dig. Du kan till exempel prova printa ut attributen, sätta in pengar och ta ut pengar. 

In [160]:
class BankAccount:
    def __init__(self, account_holder, balance):
        self.account_holder=account_holder
        self. balance=balance
    
    def deposit(self, amount): #sätta in pengar
        if amount >0:
            new_balance = self.balance + amount
            self.balance = new_balance
            print (f'Deposit amount:{amount}. New balance:{self. balance}')
    
        else:
            print("Deposit amount must be positive.")
    
 
    
    def withdraw(self, amount): #ta ut pengar
        if amount > self.balance:
            print ('Too low balance')
        elif amount<=0:
            print ('The amount must be more than 0')
        else:
            new_balance=self.balance-amount
            self.balance=new_balance
            print (f'Withdrawal amount:{amount}. New balance::{self. balance}')
        
            

In [161]:
Svensson_account=BankAccount('Svensson', 1000)
Andersson_account=BankAccount('Andersson', 120)

In [162]:
Svensson_account.deposit(100)

Deposit amount:100. New balance:1100


In [163]:
Svensson_account.deposit(300)

Deposit amount:300. New balance:1400


In [164]:
Svensson_account.withdraw(500)

Withdrawal amount:500. New balance::900


In [165]:
Andersson_account.deposit(0)

Deposit amount must be positive.


In [166]:
Andersson_account.withdraw(500)

Too low balance


7. Din kollega Adrian frågar dig, vad är klasser för något? Försök förklara detta för Adrian. Använd begreppen instanser, attribut samt metoder i din förklaring. 

Klasser är ett sätt att kombinera data och funktionalitet tillsammans. När vi skapar en ny klass, skapar vi en ny typ av objekt, som vi sedan kan skapa instanser av. Till exempel, om du har en klass Bok kan du använda den här klassen för att skapa flera böcker (instanser) med olika titlar, författare, genre och så vidare. Attribut lagrar information om en instans. För Bok kan attributen vara titel eller författare. Metoder är funktioner definierade i en klass som kan användas av instanser för att utföra olika uppgifter. För Bok kan metoderna vara boken användsk eller boken ligger på en hylla.

## Kapitel 7

1. Läs "PEP 8 – Style Guide for 
Python Cod"  och besvaranedanstående frågor.
 
   a) Om du arbetar i ett projekt som har ett internt dokument med "style guidelines" där vissa saker går emot vad PEP8 säger. Vad gör du då?
   
   b) På vilket språk skall kommentarer generellt sett skrivas.
   
   c) Hur skall funktioner och variabler namnges?
   
   d) Hur skall klasser namnges?
   
   e) Om du i ditt skript importerar exempelvis en modul, vart i skriptet skall koden som utför importen placeras?
   
   f) Enligt PEP8, skall man använda enkla ('my_string') eller dubbla ("my_string") citattecken för att skapa en sträng?
   
   g) Vilket av "Alternativ 1" och "Alternativ 2" nedan är det korrekta sättet att skriva koden på enligt PEP8?

In [167]:
# Alternativ 1
a1 = 5 + 2

# Alternativ 2
a2=5+2

a) Jag följer rekommendationerna för detta projekt;

b) Engelska;

c) gemener, med ord åtskilda av understreck
   
d) enstaka ord eller två ord går ihop med en stor bokstav i början av varje ord

e) Import bör vanligtvis ske på separata rader. Importer placeras alltid överst i filen, omedelbart efter eventuella modulkommentarer och doclines, och före modulens globala variabler och konstanter. Importer bör grupperas i följande ordning:
- Standard biblioteksimport.
- Länkad import av tredjepartsapplikationer.
- Lokal import specifik för biblioteket/den lokala applikationen.

Vi måste placera en tom rad mellan varje importgrupp.

f) ja, båda är rätt 

g) alternativ 1



2. I "PEP20 - The Zen of Python" så står det:
   
   a) "Explicit is better than implicit." Vad tror du det innebär? Kan du exemplifiera?
   
   b) "Simple is better than complex." Vad tror du det innebär? Kan du exemplifiera? 

a)  det är bättre att vara tydlig och direkt;

b) att prioritera  enkla lösningar framför mer komplexa när det är möjligt.

In [168]:
ord_lista = ["äpple", "banan", "äpple", "apelsin", "banan"]

räkna = {}
for ord in ord_lista:
    if ord in räkna:
        räkna[ord] += 1
    else:
        räkna[ord] = 1

In [169]:
from collections import Counter

ord_lista = ["äpple", "banan", "äpple", "apelsin", "banan"]
räkna = Counter(ord_lista)

3. Du och din nya kollega har en fikapaus och han säger följande: "Det känns krångligt att vi alla skall behöva följa konventioner och standarder. Är det inte lättare om alla bara skriver kod på det sättet som de vill, sålänge som den fungerar"? Vad svarar du? 

Om alla följer samma konventioner blir det mycket enklare för andra att läsa och förstå din kod.  Om alla följer olika stilar blir det svårt och tidskrävande att samarbeta. 

## Kapitel 8

1. Din kollega Johanna frågar dig:
   
   a) Vad är syntax errors?
   
   b) Varför skulle man vilja "fånga exceptions" i ett program och inte bara låta programmet stanna vid fel?
   
   c) Varför skulle man vilja "lyfta exceptions" i ett program? 

a) Syntaxfel uppstår när koden är skriven på ett sätt som Python inte förstår. 

b) Istället för att programmet kraschar, visas ett meddelande.

c) Man gör detta för att tydligt markera att ett problem har uppstått som behöver hanteras, antingen där felet lyfts eller högre upp i kodens anropskedja. 

2. a) Förklara vad nedanstående kod gör.

   b) Generellt sett, vad är poängen med att använda "else"?  

In [170]:
def convert_string_to_int(string):
    try:
        int(string)
    except ValueError:
        return "Invalid input, cannot convert to integer."
    else:
        return int(string)

print(convert_string_to_int("314")) 
print(convert_string_to_int("abc"))  

314
Invalid input, cannot convert to integer.


1) Koden definierar en funktion som försöker konvertera en sträng till ett heltal (integer).
2) "else" körs bara efter "try" har inget fel och tydligt separera fall när koden fungerar korrekt från fel. 

3. Skriv ett kodexempel där du fångar en exception. Endast din kreativitet sätter gränser. 

Inträdet till museet kostar 100 kronor. Vid ingången finns en automat som endast tar emot sedlar på 50 och 100 kronor. Automaten ger inte växel tillbaka, men returnerar överflödiga sedlar om någon stoppar in mer än vad som behövs.

Vi behöver att automaten vid varje inmatning av en sedel ökar summan och när summan når 100 kronor ger meddelandet "Vänligen, stig in".

Om den insatta summan är mindre än 100 kronor, ska automaten skriva ut ett meddelande "Vänligen, lägg till det saknade beloppet på __ kronor".

Om en sedel av annan valör stoppas in, ska automaten skriva ut ett felmeddelande "Automaten accepterar endast sedlar på 50 och 100 kronor".

In [171]:
class MuseumAutomat:
    def __init__(self):
        self.total_summa = 0
        
    def insert_banknote (self, banknote):
        try: 
            if banknote not in [50, 100]: 
                raise ValueError("Automaten accepterar endast sedlar på 50 och 100 kronor")
           
        except ValueError as e:
            print(e)
            
        else:
           
            self.total_summa += banknote
            print (self.total_summa)
           
            self.handle_amount() #call method handle_amount
            
    def handle_amount (self):
        if self.total_summa == 100:
            print("Vänligen, stig in.")
            self.total_summa = 0  # Reset the total after the visitor has gained entry
            
        elif self.total_summa < 100:
            missing_amount = 100 - self.total_summa
            print(f"Vänligen, lägg till det saknade beloppet på {missing_amount} kronor.")
        
        else:
            excess = self.total_summa - 100
            print(f"Vänligen, stig in. Här är din överskjutande summa: {excess} kronor.")
            self.total_summa = 0  # Reset the total after the visitor has gained entry
        

In [172]:
automat = MuseumAutomat()

In [173]:
automat.insert_banknote(50) 

50
Vänligen, lägg till det saknade beloppet på 50 kronor.


In [174]:
automat.insert_banknote(100) 

150
Vänligen, stig in. Här är din överskjutande summa: 50 kronor.


In [175]:
automat.insert_banknote(50)

50
Vänligen, lägg till det saknade beloppet på 50 kronor.


In [176]:
automat.insert_banknote(50)

100
Vänligen, stig in.


In [177]:
automat.insert_banknote(20)

Automaten accepterar endast sedlar på 50 och 100 kronor


In [178]:
automat.insert_banknote(100)

100
Vänligen, stig in.


4. Skapa en funktion "add_two_small_numbers" som adderar två tal. Om något av talen är större än 100 så skall du lyfta en exception och skriva ut meddelandet "both numbers must be smaller than or equal to 100". 

In [179]:
def add_twa_small(a, b):
    try:
        if a > 100 or b > 100:
            raise ValueError("Both numbers must be smaller than or equal to 100")
           
    except ValueError as e:
        print(e)
        
    else:
        print (f'Summan av a och b är lika {a + b}')

In [180]:
add_twa_small(45,120)

Both numbers must be smaller than or equal to 100


In [181]:
add_twa_small(45,15)

Summan av a och b är lika 60


5. Din kollega, som är en skicklig programmerare, brukar innan hon försöker göra ett perfekt fungerande program testa olika ideér för att undersöka och lära sig mer om det problem hon försöker lösa. Nedan ser du ett av hennes skript som gjorts i syfte att undersöka och lära sig mer. Förklara vad det är hon gjort. 

Hon försöker att skapa Exception-klasser som representerar och specificera möjliga exception  

In [182]:
# Checking which exception is raised
try:
    5 + "Python is fun!"
except Exception as exception_instance:
    print(type(exception_instance))
    print(exception_instance)

<class 'TypeError'>
unsupported operand type(s) for +: 'int' and 'str'


In [183]:
#Python tillåter inte att man adderar en int och en str direkt. Resultaten är ett exception som har "class 'TypeError'" 
#så  problemet är med datatyper i en operation.

In [184]:
# Checking which exception is raised
try:
    5/0
except Exception as exception_instance:
    print(type(exception_instance))
    print(exception_instance)

<class 'ZeroDivisionError'>
division by zero


In [185]:
#Matematiskt är det ett fel att dividera med noll så Python visar som resultatet ett exception som har "class 'ZeroDivisionError'" 
#så  problemet är omöjlighet att dividera med noll.

In [186]:
def add_two_numbers(a, b):
    try:
        return(a/b)
    except TypeError:
        print("Both arguments must be numbers.")
    except ZeroDivisionError:
        print("Division by zero is not defined.")   

In [187]:
# Testing so the functionality is as expected
print(add_two_numbers(5, 2))
print(add_two_numbers(5, "hello"))
print(add_two_numbers(5, 0))

2.5
Both arguments must be numbers.
None
Division by zero is not defined.
None


Efter hon har definierat klasser för exception skapar hon en funktion add_two_numbers med except-klausuler som kommer att visa ett tydligare meddelande till användaren för exception av klasserna ZeroDivisionError' och 'TypeError'

## Kapitel 9

1. Varför är det generellt sett en god idé att bygga upp tester för sin kod? 

Tester kan hjälpa: 
- att fånga fel tidigt;
- att säkerställa att ändringarna inte orsakar oväntade problem i andra delar av systemet;
- att förstå  snabbt vad koden är tänkt att göra genom att läsa testerna. 

2. Nedan ser du funktionen multiply_two_numbers. Använd "doctest" modulen för att implementera några test för funktionen "multiply_two_numbers". 
Den läsare som önskar kan även implementera testen i modulen "pytest". 

In [188]:
def multiply_two_numbers(a, b):
    """Return the product of a and b."""
    return a*b

In [189]:
def multiply_two_numbers(a, b):
    """
    Return the product of a and b.

    Examples:
    >>> multiply_two_numbers(2, 3)
    6
    >>> multiply_two_numbers(-1, 5)
    -5
    >>> multiply_two_numbers(0, 100)
    0
    """
    return a * b

[Test för funktionen "multiply_two_numbers"](multiply.py)

![Testresultatet](kap9_f2.png)

3. Skriv någon kod eller använd någon kod som du till exempel skrivit i tidigare kapitels uppgifter och implementera några test. Använd "doctest" och/eller "pytest" modulen. 

In [191]:
def add_or_multiply_pytest(a, b, operation):
    if operation == "add":
        return a + b
    elif operation == "multiply":
        return a * b
    else:
        return 'Choose either "add" or "multiply"'

[Test för funktionen "add_or_multiply_pytest"](kap9_f3.py)

![Testresultatet](kap9_f3.png)

## Kapitel 10

3. Bygg en egen modul som till exempel innehåller två funktioner, importera därefter denna modulen i ett annat skript och prova använd de två funktionerna. Notera att modulen du bygger skall skrivas i ett "vanligt skript" (en .py fil) och inte i en jupyter notebook fil (.ipynb). Du kan däremot importera modulen (.py filen) i en jupyter notebook (.ipynb fil).
  
   Om du önskar importera en jupyter notebook (.ipynb fil) till en annan jupyter notebook (.ipynb fil) så kan du kolla igenom följande smidiga paket: https://pypi.org/project/import-ipynb/ .

[Modul](module_kap10_f3.py)

In [192]:
#The function for calculating the area of a rectangle
def area(a, b):
    """Rreturn the area of a rectangle with sides a and b"""
    return a * b

#The function for calculating the perimetr of a rectangle
def perimeter(a, b):
    """Rreturn the perimeter of a rectangle with sides a and b."""
    return 2*a + 2*b

[Script](script_kap10_f3.py)

In [193]:
# Import of module
from module_kap10_f3 import area, perimeter

# Use the functions
area_result = area(2, 3)
perimeter_result = perimeter(2, 3)

print(f"Area: {area_result}")
print(f"Perimeter: {perimeter_result}")

Area: 6
Perimeter: 10


![Test](kap10_f3.png)

## Kapital 11

1. Varför kan loggfiler med information vara viktiga för exempelvis företag och organisationer? 

Loggfilen är utformad för att lagra detaljerad information om hur systemet fungerar. Detta gör att man snabbt kan identifiera fel och analysera dem.

2. Vad menas med att det finns olika nivåer för logg-meddelanden? 

Loggin-modulen har fem nivåer på meddelanden: 
- DEBUG (detaljerad information);
- INFO (bekräftelser att programmet fungerar som det ska);
- WARNING (något oväntat hänt alllr är på väg att hända);
- ERROR (ett fel);
- CRITICAL (ett allvarligt fel, programmet har eventuellt stannat).

Dessa nivåer vissar grad av allvarlighet eller betydelse som en händelse har när den loggas och gör det möjligt att filtrera och prioritera loggar baserat på deras relevans eller kritikalitet.


3. Skapa ett valfritt program där du också använder dig av logging modulen. 

En funktion som beräknar max_100 = a * (b + c / d) - 33, loggar ett meddelande beroende på resultatet och hanterar specialfallet där d=0 med en kritisk felhantering.

In [67]:
import logging
logger=logging.getLogger()

In [71]:
logging.basicConfig(
    format='[%(asctime)s][%(levelname)s] %(message)s',
    level=logging.DEBUG,
    filename='limit.log'
)

In [197]:

def calculate_limit(a, b, c, d):
    try:
        if d==0:
           #Crittical log-message
            logging.critical("Division by zero detected! 'd' cannot be 0.")
            return None
        
        limit = a * (b + c / d) - 33  # Calculete max_100
        logging.debug(f"limit: {limit}")
        
         # Log-level based on the max_100
        if 0 < limit < 95:
            logging.info(f"INFO: limit value is in normal range: {limit}")
        elif 95 <= limit <= 100:
            logging.warning(f"WARNING: limit is in the warning range: {limit}")
        elif limit > 100:
            logging.error(f"ERROR: limit value exceeds the limit: {limit}")
        elif limit <= 0:
            logging.error (f"ERROR: max_100 value less than the limit 0: {limit}")

        return limit
    except Exception as e:
       # Log an error if anything else goes wrong
        logging.error(f"An error occurred during calculation: {e}")
        return None
    

In [198]:
calculate_limit(2, 50, 10, 5)  # Normal range
calculate_limit(2, 55, 10, 1)  # Warning
calculate_limit(2, 100, 50, 1)  # Error
calculate_limit(2, 50, 10, 0)  # Critical (d=0)
calculate_limit(2, -50, 10, 1) # Error

DEBUG:root:limit: 71.0
INFO:root:INFO: limit value is in normal range: 71.0
DEBUG:root:limit: 97.0
DEBUG:root:limit: 267.0
ERROR:root:ERROR: limit value exceeds the limit: 267.0
CRITICAL:root:Division by zero detected! 'd' cannot be 0.
DEBUG:root:limit: -113.0
ERROR:root:ERROR: max_100 value less than the limit 0: -113.0


-113.0

[Modul](kap11_f3.py)

[log_message](limit.log)